In [1]:
import sys
import os
sys.path.append(os.path.join(os.getcwd(),"scripts"))  #  import issues

import numpy as np
import matplotlib.pyplot as plt
import copy
import torch
import datetime
import time 

# import scripts
from scripts.api_helper import transformer
from scripts.api_helper import fake_api

from scripts.api_helper import api_utils
from scripts.api_helper import portfolio_constructor
from scripts.exp_helper import exp_helper
from scripts.Botorch_opt import bayesian_optimiser
from scripts.function_slicer import slicer
from scripts.gp_evaluator import gp_evaluation
from scripts import optutil

dtype = torch.float

In [2]:
"""
Experiment:

1. retrieve portfolio
2. for each portfolio do:

    i. propose 400 random queries
    ii. GP evaluation 
    iii. pick best kernel + tuning method

    for each acquisition function:
        run optimisation; 
        record descent path;
Arg:
    T: mins for hard termination
    cobdate: previous business date
    data_folder: the path for which data are saving; this will OVERWRITE existing content

"""

'\nExperiment:\n\n1. retrieve portfolio\n2. for each portfolio do:\n\n    i. propose 400 random queries\n    ii. GP evaluation \n    iii. pick best kernel + tuning method\n\n    for each acquisition function:\n        run optimisation; \n        record descent path;\nArg:\n    T: mins for hard termination\n    cobdate: previous business date\n    data_folder: the path for which data are saving; this will OVERWRITE existing content\n\n'

In [3]:
cobdate = datetime.date(2020, 8, 20)
data_folder = "experiment4"

In [4]:
#  hyper-parameter

T = 20  # hard termination mins
runs = 1 # run of experiment 
N, q = 400, 10     # training samples for GP evaluation
portfolio_fil_name = "all_portfolios"

# gp; includes "MA", "SE", "RQ", "LR", "PO"
gp_name,gp_params = "MA",{"nu":2.5,           # hyper-parameter for matern kernel
                          "mode": "raw",   # kernel design idea
                         "opt":"ADAM", # opt for MLE; includes: quasi_newton, ADAM
                          "epochs":128, # if chosen ADAM as opt
                          "lr":1e-1, # learning rate for ADAM
                         }
# q-parallelism (if use analytical acq_func, q must be 1)
batch_size = 1

hyper_params = { 
    "acq_name" : "UCB",          # acqu func; includes: "EI", "UCB", "qEI", "qUCB", "qKG", "MES"
    "N_start": 32,               # number of starts for multi-start SGA
    "raw_samples" :512,          # heuristic initialisation 
    "N_MC_sample" : 256,         # number of samples for Monte Carlo simulation
    "num_fantasies": 128,        # number of fantasies used by KG
    "beta":1.,                   # used by UCB/qUCB
               }

In [5]:
#  retrieve portfolio
full_path = os.path.join(data_folder, portfolio_fil_name)
with open(full_path + ".pt", "rb") as f:
    data = torch.load(f)
    f.close()
tables = [i[0] for i in data]
margin_params = [i[1] for i in data]
portfolios = portfolio_constructor.construct_portfolio(tables, folder_name = "")

not saving unless provide a file_name


In [9]:
schedule = datetime.datetime(2020, 8 , 28 ,18, 0, 0, 0)
datetime.datetime.now() < schedule

True

In [10]:
while True:
    if datetime.datetime.now() < schedule:
        time.sleep(600)
    else:
        print(datetime.datetime.now())
        break

2020-08-21 18:01:14.418651


##### experiment

In [ ]:
for i, portfolio in enumerate(portfolios):
    
    t1 = time.time()
    
    #  prepare api && init samples
    wrap, x0, y0, m0 = api_utils.create_start_position(optutil.OptimisationWrap, cobdate, 
                                                        portfolio,  margin_params[i], fixed_start=True)

    @api_utils.transform
    def api(x):
        return wrap.f(x)
    print("starting point & reward", x0, y0)
    print(f"initial margin {m0:,.0f}")

    #  random queries for training GP
    X, y = slicer.static_random_query(x0.size(-1), api, m0, N, q)

    #  gp evaluation
    records = gp_evaluation.evaluation(X, y)
    kernel, mode, opt, _, _ = gp_evaluation.result_and_save(records, data_folder, i)
    gp_name, gp_params["mode"], gp_params["opt"] = kernel, mode, opt
    print("")
    print(f"gp is {kernel}, {mode}, {opt}")

    #  experiment 1: exploration- exploitation trade-off
    for _ in range(3):
        try:
            best_beta = exp_helper.exp1(gp_name, gp_params, hyper_params, data_folder, i, T, x0, y0, m0, api, runs)
            print("")
            print(f"best beta {best_beta:.1f}")
            hyper_params["beta"] = best_beta
        except:
            time.sleep(10)
        else:
            break

    #  experiment 2: UCB_EI && their parallelism
    for _ in range(3):
        try:
            exp_helper.exp2(gp_name, gp_params, hyper_params, data_folder, i, T, x0, y0, m0, api, runs, best_beta)
        except:
            time.sleep(10)
        else:
            break
    
    #  experiment 3: KG
    for _ in range(3):
        try:
            exp_helper.exp3(gp_name, gp_params, hyper_params, data_folder, i, T, x0, y0, m0, api, runs)
        except:
            time.sleep(10)
        else:
            break
    
    #  experiment 4: warmups
    for _ in range(3):
        try:
            exp_helper.exp4(gp_name, gp_params, hyper_params, data_folder, i, T, x0, y0, m0, api, runs)
    except:
        time.sleep(10)
    else:
        break
    
    #  time 
    t2 = time.time()
    print(f"this run took {(t2-t1)/3600:.1f} hrs")

$208,053,144, buffer is: $10,379,852
starting point & reward tensor([[0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000,
         0.5000, 0.5000, 0.5000, 0.5000, 0.5000]]) tensor([[-1.]])
initial margin 208,053,144
0 queries are made
50 queries are made
100 queries are made
150 queries are made
200 queries are made
250 queries are made
300 queries are made
350 queries are made


c:\python\python37\lib\site-packages\gpytorch\utils\cholesky.py:44: NumericalWarning: A not p.d., added jitter of 1e-06 to the diagonal
  warnings.warn(f"A not p.d., added jitter of {jitter_new} to the diagonal", NumericalWarning)
c:\python\python37\lib\site-packages\gpytorch\utils\cholesky.py:44: NumericalWarning: A not p.d., added jitter of 9.999999999999999e-06 to the diagonal
  warnings.warn(f"A not p.d., added jitter of {jitter_new} to the diagonal", NumericalWarning)
c:\python\python37\lib\site-packages\gpytorch\lazy\lazy_tensor.py:1330: NumericalWarning: Runtime Error when computing Cholesky decomposition: cholesky_cpu: For batch 0: U(2,2) is zero, singular U.. Using RootDecomposition.
  NumericalWarning,
c:\python\python37\lib\site-packages\gpytorch\lazy\lazy_tensor.py:1330: NumericalWarning: Runtime Error when computing Cholesky decomposition: cholesky_cpu: For batch 0: U(4,4) is zero, singular U.. Using RootDecomposition.
  NumericalWarning,
c:\python\python37\lib\site-packag


gp is RQ, pro, ADAM
starting first experiment
Iter 128 - Loss: 0.047
time step 1, drop -23.32%; min $256,580,624
Iter 128 - Loss: -0.730
time step 2, drop -0.22%; min $208,513,600
Iter 128 - Loss: -1.493
time step 3, drop -2.88%; min $214,046,848
Iter 128 - Loss: -1.664
time step 4, drop -2.80%; min $213,868,528
Iter 128 - Loss: -1.770
time step 5, drop -1.24%; min $210,623,680
Iter 128 - Loss: -1.930
time step 6, drop -0.30%; min $208,675,568
Iter 128 - Loss: -2.079
time step 7, drop -0.42%; min $208,933,520
Iter 128 - Loss: -2.214
time step 8, drop -0.47%; min $209,029,232
Iter 128 - Loss: -2.321
time step 9, drop -0.76%; min $209,640,240
Iter 128 - Loss: -2.404
time step 10, drop -0.97%; min $210,070,768
Iter 128 - Loss: -2.474
time step 11, drop -0.17%; min $208,402,672
Iter 128 - Loss: -2.525
time step 12, drop -0.32%; min $208,724,048
Iter 128 - Loss: -2.581
time step 13, drop -0.71%; min $209,526,992
Iter 128 - Loss: -2.627
time step 14, drop -0.98%; min $210,087,952
Iter 128 -

Iter 128 - Loss: -3.459
time step 121, drop -0.09%; min $208,232,064
Iter 128 - Loss: -3.461
time step 122, drop -0.11%; min $208,273,504
Iter 128 - Loss: -3.462
time step 123, drop -0.11%; min $208,289,872
Iter 128 - Loss: -3.464
time step 124, drop -0.12%; min $208,295,600
Iter 128 - Loss: -3.465
time step 125, drop -0.33%; min $208,748,896
Iter 128 - Loss: -3.467
time step 126, drop -0.10%; min $208,264,400
Iter 128 - Loss: -3.468
time step 127, drop -0.08%; min $208,217,696
Iter 128 - Loss: -3.469
time step 128, drop -0.11%; min $208,274,512
Iter 128 - Loss: -3.471
time step 129, drop -0.11%; min $208,287,472
Iter 128 - Loss: -3.472
time step 130, drop -0.41%; min $208,908,112
Iter 128 - Loss: -3.473
time step 131, drop -0.47%; min $209,033,648
Iter 128 - Loss: 0.047
time step 1, drop -13.71%; min $236,572,560
Iter 128 - Loss: -1.261
time step 2, drop -0.36%; min $208,793,216
Iter 128 - Loss: -1.809
time step 3, drop -4.60%; min $217,626,416
Iter 128 - Loss: -1.864
time step 4, dro

Iter 128 - Loss: -3.377
time step 111, drop -0.08%; min $208,223,920
Iter 128 - Loss: -3.379
time step 112, drop 0.07%; min $207,907,760
Iter 128 - Loss: -3.382
time step 113, drop -0.48%; min $209,051,984
Iter 128 - Loss: -3.383
time step 114, drop -0.45%; min $208,991,408
Iter 128 - Loss: -3.384
time step 115, drop 0.06%; min $207,934,080
Iter 128 - Loss: -3.386
time step 116, drop 0.16%; min $207,723,040
Iter 128 - Loss: -3.387
time step 117, drop 0.29%; min $207,457,920
Iter 128 - Loss: -3.388
time step 118, drop 0.20%; min $207,630,272
Iter 128 - Loss: 0.047
time step 1, drop -20.23%; min $250,148,656
Iter 128 - Loss: -0.872
time step 2, drop -1.88%; min $211,967,424
Iter 128 - Loss: -1.387
time step 3, drop -1.64%; min $211,462,544
Iter 128 - Loss: -1.546
time step 4, drop -3.41%; min $215,151,232
Iter 128 - Loss: -1.658
time step 5, drop -3.15%; min $214,614,800
Iter 128 - Loss: -1.754
time step 6, drop -1.02%; min $210,177,936
Iter 128 - Loss: -1.849
time step 7, drop -11.93%; 

...

time step 2, drop -4.76%; min $152,748,992
Iter 128 - Loss: -0.899
time step 3, drop -12.24%; min $163,646,400
Iter 128 - Loss: -1.003
time step 4, drop -9.21%; min $159,230,832
Iter 128 - Loss: -1.105
time step 5, drop -12.87%; min $164,569,664
Iter 128 - Loss: -1.167
time step 6, drop -4.53%; min $152,402,432
Iter 128 - Loss: -1.244
time step 7, drop -18.90%; min $173,360,240
Iter 128 - Loss: -1.162
time step 8, drop -5.30%; min $153,532,432
Iter 128 - Loss: -1.234
time step 9, drop -3.95%; min $151,565,168
Iter 128 - Loss: -1.293
time step 10, drop -8.75%; min $158,556,208
Iter 128 - Loss: -1.336
time step 11, drop -5.63%; min $154,016,048
Iter 128 - Loss: -1.388
time step 12, drop -1.64%; min $148,194,848
Iter 128 - Loss: -1.424
time step 13, drop -8.97%; min $158,877,088
Iter 128 - Loss: -1.446
time step 14, drop -1.93%; min $148,613,632
Iter 128 - Loss: -1.484
time step 15, drop -9.15%; min $159,149,984
Iter 128 - Loss: -1.487
time step 16, drop -3.66%; min $151,135,920
Iter 128 

Iter 128 - Loss: -1.423
time step 17, drop -0.60%; min $146,670,560
Iter 128 - Loss: -1.470
time step 18, drop -1.73%; min $148,318,992
Iter 128 - Loss: -1.510
time step 19, drop -0.36%; min $146,333,312
Iter 128 - Loss: -1.596
time step 20, drop -0.31%; min $146,260,208
Iter 128 - Loss: -1.624
time step 21, drop -1.90%; min $148,566,128
Iter 128 - Loss: -1.649
time step 22, drop -0.24%; min $146,145,408
Iter 128 - Loss: -1.700
time step 23, drop -0.56%; min $146,622,000
Iter 128 - Loss: -1.755
time step 24, drop 0.26%; min $145,429,920
Iter 128 - Loss: -1.820
time step 25, drop -0.34%; min $146,304,576
Iter 128 - Loss: -1.877
time step 26, drop -0.92%; min $147,145,696
Iter 128 - Loss: -1.915
time step 27, drop -0.87%; min $147,069,536
Iter 128 - Loss: -1.959
time step 28, drop -0.59%; min $146,659,504
Iter 128 - Loss: -2.005
time step 29, drop 0.07%; min $145,698,656
Iter 128 - Loss: -2.047
time step 30, drop 0.62%; min $144,901,744
Iter 128 - Loss: -2.085
time step 31, drop -0.21%; 

Iter 128 - Loss: -1.880
time step 29, drop 0.02%; min $145,777,264
Iter 128 - Loss: -1.899
time step 30, drop -0.81%; min $146,989,664
Iter 128 - Loss: -1.912
time step 31, drop -0.44%; min $146,448,160
Iter 128 - Loss: -1.933
time step 32, drop -0.32%; min $146,264,800
Iter 128 - Loss: -1.945
time step 33, drop 0.28%; min $145,396,640
Iter 128 - Loss: -1.957
time step 34, drop -0.61%; min $146,692,960
Iter 128 - Loss: -1.983
time step 35, drop -0.41%; min $146,406,912
Iter 128 - Loss: -2.004
time step 36, drop 0.40%; min $145,217,264
Iter 128 - Loss: -2.030
time step 37, drop -0.37%; min $146,344,192
Iter 128 - Loss: -2.052
time step 38, drop 0.25%; min $145,440,576
Iter 128 - Loss: -2.075
time step 39, drop 0.17%; min $145,555,680
Iter 128 - Loss: -2.092
time step 40, drop 0.21%; min $145,491,584
Iter 128 - Loss: -2.112
time step 41, drop 0.54%; min $145,012,448
Iter 128 - Loss: -2.138
time step 42, drop -0.29%; min $146,223,264
Iter 128 - Loss: -2.156
time step 43, drop 0.98%; min $

Iter 128 - Loss: -1.711
time step 13, drop -3.95%; min $151,568,192
Iter 128 - Loss: -1.721
time step 14, drop -0.86%; min $147,054,784
Iter 128 - Loss: -1.793
time step 15, drop -0.07%; min $145,901,456
Iter 128 - Loss: -1.819
time step 16, drop -2.65%; min $149,668,192
Iter 128 - Loss: -1.858
time step 17, drop -3.31%; min $150,631,824
Iter 128 - Loss: -1.885
time step 18, drop -0.98%; min $147,230,848
Iter 128 - Loss: -1.890
time step 19, drop -1.32%; min $147,730,304
Iter 128 - Loss: -1.837
time step 20, drop -3.59%; min $151,031,392
Iter 128 - Loss: -1.837
time step 21, drop -3.28%; min $150,584,928
Iter 128 - Loss: -1.846
time step 22, drop 0.51%; min $145,057,008
Iter 128 - Loss: -1.886
time step 23, drop 0.39%; min $145,235,072
Iter 128 - Loss: -1.934
time step 24, drop 0.73%; min $144,744,688
Iter 128 - Loss: -1.946
time step 25, drop -1.14%; min $147,467,408
Iter 128 - Loss: -1.961
time step 26, drop -0.56%; min $146,624,272
Iter 128 - Loss: -1.993
time step 27, drop -1.51%; 

Iter 128 - Loss: -2.626
time step 61, drop 0.84%; min $144,573,376
Iter 128 - Loss: -2.641
time step 62, drop 1.37%; min $143,808,544
Iter 128 - Loss: -2.656
time step 63, drop 1.31%; min $143,892,288
Iter 128 - Loss: -2.670
time step 64, drop 1.17%; min $144,094,576
Iter 128 - Loss: -2.685
time step 65, drop 1.41%; min $143,743,200
Iter 128 - Loss: -2.698
time step 66, drop 1.42%; min $143,726,192
Iter 128 - Loss: -2.711
time step 67, drop 1.40%; min $143,761,776
Iter 128 - Loss: -2.723
time step 68, drop 1.37%; min $143,797,344
Iter 128 - Loss: -2.736
time step 69, drop 1.34%; min $143,841,968
Iter 128 - Loss: -2.748
time step 70, drop 1.14%; min $144,146,128
Iter 128 - Loss: -2.760
time step 71, drop 1.17%; min $144,098,224
Iter 128 - Loss: -2.772
time step 72, drop 1.06%; min $144,255,136
Iter 128 - Loss: -2.783
time step 73, drop 1.22%; min $144,028,656
Iter 128 - Loss: -2.794
time step 74, drop 0.98%; min $144,376,112
Iter 128 - Loss: -2.805
time step 75, drop 1.09%; min $144,209

In [ ]:
i

In [10]:
#  retrieve portfolio
full_path = os.path.join(data_folder, portfolio_fil_name)
with open(full_path + ".pt", "rb") as f:
    data = torch.load(f)
    f.close()
tables = [i[0] for i in data]
margin_params = [i[1] for i in data]
portfolios = portfolio_constructor.construct_portfolio(tables, folder_name = "")

not saving unless provide a file_name


In [11]:
portfolios = portfolios[2:]
len(portfolios)

6

In [ ]:
for i, portfolio in enumerate(portfolios):
    
    t1 = time.time()
    
    #  prepare api && init samples
    wrap, x0, y0, m0 = api_utils.create_start_position(optutil.OptimisationWrap, cobdate, 
                                                        portfolio,  margin_params[i+4], fixed_start=True)

    @api_utils.transform
    def api(x):
        return wrap.f(x)
    print("starting point & reward", x0, y0)
    print(f"initial margin {m0:,.0f}")

    #  random queries for training GP
    X, y = slicer.static_random_query(x0.size(-1), api, m0, N, q)

    #  gp evaluation
    records = gp_evaluation.evaluation(X, y)
    kernel, mode, opt, _, _ = gp_evaluation.result_and_save(records, data_folder, i+4)
    gp_name, gp_params["mode"], gp_params["opt"] = kernel, mode, opt
    print("")
    print(f"gp is {kernel}, {mode}, {opt}")

    #  experiment 1: exploration- exploitation trade-off
    for _ in range(3):
        try:
            best_beta = exp_helper.exp1(gp_name, gp_params, hyper_params, data_folder, i+4, T, x0, y0, m0, api, runs)
            print("")
            print(f"best beta {best_beta:.1f}")
        except:
            time.sleep(10)
        else:
            break

    #  experiment 2: UCB_EI && their parallelism
    for _ in range(3):
        try:
            exp_helper.exp2(gp_name, gp_params, hyper_params, data_folder, i+4, T, x0, y0, m0, api, runs, best_beta)
        except:
            time.sleep(10)
        else:
            break
    
    #  experiment 3: KG
    for _ in range(3):
        try:
            exp_helper.exp3(gp_name, gp_params, hyper_params, data_folder, i+4, T, x0, y0, m0, api, runs)
        except:
            time.sleep(10)
        else:
            break
    
    #  experiment 4: warmups
    for _ in range(3):
        try:
            exp_helper.exp4(gp_name, gp_params, hyper_params, data_folder, i+4, T, x0, y0, m0, api, runs)
    except:
        time.sleep(10)
    else:
        break
    
    #  time 
    t2 = time.time()
    print(f"this run took {(t2-t1)/3600:.1f} hrs")

$161,104,472, buffer is: $2,770,154
starting point & reward tensor([[0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000,
         0.5000, 0.5000, 0.5000, 0.5000, 0.5000]]) tensor([[-1.]])
initial margin 161,104,472
0 queries are made
50 queries are made
100 queries are made
150 queries are made
200 queries are made
250 queries are made
300 queries are made
350 queries are made


c:\python\python37\lib\site-packages\gpytorch\utils\cholesky.py:44: NumericalWarning: A not p.d., added jitter of 9.999999999999999e-06 to the diagonal
  warnings.warn(f"A not p.d., added jitter of {jitter_new} to the diagonal", NumericalWarning)
c:\python\python37\lib\site-packages\gpytorch\utils\cholesky.py:44: NumericalWarning: A not p.d., added jitter of 1e-06 to the diagonal
  warnings.warn(f"A not p.d., added jitter of {jitter_new} to the diagonal", NumericalWarning)
c:\python\python37\lib\site-packages\gpytorch\utils\cholesky.py:44: NumericalWarning: A not p.d., added jitter of 9.999999999999999e-06 to the diagonal
  warnings.warn(f"A not p.d., added jitter of {jitter_new} to the diagonal", NumericalWarning)
c:\python\python37\lib\site-packages\gpytorch\lazy\lazy_tensor.py:1330: NumericalWarning: Runtime Error when computing Cholesky decomposition: cholesky_cpu: For batch 0: U(4,4) is zero, singular U.. Using RootDecomposition.
  NumericalWarning,
c:\python\python37\lib\site-pac


gp is RQ, pro, ADAM
starting first experiment
Iter 128 - Loss: 0.047
time step 1, drop -62.31%; min $261,480,768
Iter 128 - Loss: 0.253
time step 2, drop 0.15%; min $160,856,976
Iter 128 - Loss: -0.655
time step 3, drop 0.21%; min $160,760,160
Iter 128 - Loss: -0.862
time step 4, drop -0.65%; min $162,153,504
Iter 128 - Loss: -1.064
time step 5, drop 0.37%; min $160,506,832
Iter 128 - Loss: -1.222
time step 6, drop -0.49%; min $161,894,384
Iter 128 - Loss: -1.348
time step 7, drop 0.27%; min $160,673,200
Iter 128 - Loss: -1.460
time step 8, drop -0.68%; min $162,193,296
Iter 128 - Loss: -1.542
time step 9, drop -0.04%; min $161,173,136
Iter 128 - Loss: -1.628
time step 10, drop -2.71%; min $165,471,232
Iter 128 - Loss: -1.659
time step 11, drop -1.04%; min $162,774,192
Iter 128 - Loss: -1.687
time step 12, drop -1.33%; min $163,239,312
Iter 128 - Loss: -1.723
time step 13, drop -0.52%; min $161,940,688
Iter 128 - Loss: -1.787
time step 14, drop 0.13%; min $160,892,432
Iter 128 - Loss:

c:\python\python37\lib\site-packages\gpytorch\utils\cholesky.py:44: NumericalWarning: A not p.d., added jitter of 9.999999999999999e-06 to the diagonal
  warnings.warn(f"A not p.d., added jitter of {jitter_new} to the diagonal", NumericalWarning)


Iter 128 - Loss: -3.164
time step 72, drop 1.12%; min $159,292,928
Iter 128 - Loss: -3.170
time step 73, drop 1.27%; min $159,061,296
Iter 128 - Loss: -3.175
time step 74, drop 0.87%; min $159,709,408


...

time step 2, drop -0.55%; min $161,995,104
Iter 128 - Loss: -0.746
time step 3, drop 0.25%; min $160,703,456
Iter 128 - Loss: -0.882
time step 4, drop -1.12%; min $162,909,504
Iter 128 - Loss: -1.015
time step 5, drop -0.02%; min $161,138,896
Iter 128 - Loss: -1.159
time step 6, drop -3.09%; min $166,086,624
Iter 128 - Loss: -1.213
time step 7, drop -0.47%; min $161,869,520
Iter 128 - Loss: -1.334
time step 8, drop 0.85%; min $159,741,872
Iter 128 - Loss: -1.435
time step 9, drop -0.62%; min $162,100,976
Iter 128 - Loss: -1.471
time step 10, drop -2.85%; min $165,692,464
Iter 128 - Loss: -1.491
time step 11, drop 0.48%; min $160,323,936
Iter 128 - Loss: -1.553
time step 12, drop -0.58%; min $162,046,032
Iter 128 - Loss: -1.599
time step 13, drop 0.59%; min $160,155,984
Iter 128 - Loss: -1.704
time step 14, drop -0.17%; min $161,379,104
Iter 128 - Loss: -1.724
time step 15, drop -0.33%; min $161,630,480
Iter 128 - Loss: -1.764
time step 16, drop 1.12%; min $159,305,152
Iter 128 - Loss: 

Iter 128 - Loss: -2.195
time step 46, drop -0.47%; min $161,861,536
Iter 128 - Loss: -2.210
time step 47, drop -1.08%; min $162,841,872
Iter 128 - Loss: -2.219
time step 48, drop 0.62%; min $160,098,080
Iter 128 - Loss: -2.238
time step 49, drop 0.18%; min $160,808,528
Iter 128 - Loss: -2.255
time step 50, drop -3.20%; min $166,257,632
Iter 128 - Loss: -2.253
time step 51, drop -0.06%; min $161,202,224
Iter 128 - Loss: -2.266
time step 52, drop 0.68%; min $160,014,880
Iter 128 - Loss: -2.288
time step 53, drop 0.71%; min $159,959,056
Iter 128 - Loss: -2.310
time step 54, drop 0.72%; min $159,950,400
Iter 128 - Loss: -2.332
time step 55, drop 0.87%; min $159,703,152
Iter 128 - Loss: -2.356
time step 56, drop 0.63%; min $160,083,824
Iter 128 - Loss: -2.376
time step 57, drop 1.32%; min $158,975,552
Iter 128 - Loss: -2.394
time step 58, drop 1.30%; min $159,016,752
Iter 128 - Loss: -2.413
time step 59, drop 1.20%; min $159,177,456
Iter 128 - Loss: -2.430
time step 60, drop 1.18%; min $159

c:\python\python37\lib\site-packages\gpytorch\utils\cholesky.py:44: NumericalWarning: A not p.d., added jitter of 1e-06 to the diagonal
  warnings.warn(f"A not p.d., added jitter of {jitter_new} to the diagonal", NumericalWarning)
c:\python\python37\lib\site-packages\gpytorch\utils\cholesky.py:44: NumericalWarning: A not p.d., added jitter of 9.999999999999999e-06 to the diagonal
  warnings.warn(f"A not p.d., added jitter of {jitter_new} to the diagonal", NumericalWarning)
c:\python\python37\lib\site-packages\gpytorch\lazy\lazy_tensor.py:1330: NumericalWarning: Runtime Error when computing Cholesky decomposition: cholesky_cpu: For batch 0: U(3,3) is zero, singular U.. Using RootDecomposition.
  NumericalWarning,
c:\python\python37\lib\site-packages\gpytorch\lazy\lazy_tensor.py:1330: NumericalWarning: Runtime Error when computing Cholesky decomposition: cholesky_cpu: For batch 0: U(4,4) is zero, singular U.. Using RootDecomposition.
  NumericalWarning,
c:\python\python37\lib\site-packag


gp is RQ, pro, ADAM
starting first experiment
Iter 128 - Loss: 0.047
time step 1, drop -22.16%; min $195,850,416
Iter 128 - Loss: -0.781
time step 2, drop -0.07%; min $160,427,408
Iter 128 - Loss: -1.534
time step 3, drop -1.29%; min $162,388,144
Iter 128 - Loss: -1.765
time step 4, drop -0.50%; min $161,124,304
Iter 128 - Loss: -1.974
time step 5, drop -0.03%; min $160,374,208
Iter 128 - Loss: -2.123
time step 6, drop -0.66%; min $161,380,400
Iter 128 - Loss: -2.240
time step 7, drop -1.61%; min $162,892,928
Iter 128 - Loss: -2.309
time step 8, drop -1.63%; min $162,938,768
Iter 128 - Loss: -2.339
time step 9, drop -0.69%; min $161,419,616
Iter 128 - Loss: -2.417
time step 10, drop -0.79%; min $161,590,736
Iter 128 - Loss: -2.485
time step 11, drop -0.25%; min $160,714,288
Iter 128 - Loss: -2.542
time step 12, drop -0.19%; min $160,627,008
Iter 128 - Loss: -2.597
time step 13, drop -0.59%; min $161,264,304
Iter 128 - Loss: -2.645
time step 14, drop -0.29%; min $160,787,696
Iter 128 -

time step 24, drop 0.14%; min $160,090,144
Iter 128 - Loss: -2.681
time step 25, drop 0.06%; min $160,224,320
Iter 128 - Loss: -2.700
time step 26, drop -0.05%; min $160,392,720
Iter 128 - Loss: -2.727
time step 27, drop 0.26%; min $159,905,040
Iter 128 - Loss: -2.752
time step 28, drop -0.12%; min $160,517,920
Iter 128 - Loss: -2.774
time step 29, drop -0.23%; min $160,691,072
Iter 128 - Loss: -2.797
time step 30, drop -0.69%; min $161,416,784
Iter 128 - Loss: -2.815
time step 31, drop -1.28%; min $162,375,904
Iter 128 - Loss: -2.822
time step 32, drop -0.77%; min $161,556,704
Iter 128 - Loss: -2.837
time step 33, drop 0.38%; min $159,710,832
Iter 128 - Loss: -2.856
time step 34, drop 0.44%; min $159,607,136
Iter 128 - Loss: -2.874
time step 35, drop 0.35%; min $159,751,808
Iter 128 - Loss: -2.891
time step 36, drop 0.38%; min $159,711,424
Iter 128 - Loss: -2.908
time step 37, drop 0.01%; min $160,304,256
Iter 128 - Loss: -2.925
time step 38, drop 0.08%; min $160,184,352
Iter 128 - Lo

In [ ]:
#  retrieve portfolio
full_path = os.path.join(data_folder, portfolio_fil_name)
with open(full_path + ".pt", "rb") as f:
    data = torch.load(f)
    f.close()
tables = [i[0] for i in data]
margin_params = [i[1] for i in data]
portfolios = portfolio_constructor.construct_portfolio(tables, folder_name = "")

In [ ]:
portfolio = portfolios[1]

In [ ]:
#  prepare api && init samples
wrap, x0, y0, m0 = api_utils.create_start_position(optutil.OptimisationWrap, cobdate, 
                                                    portfolio,  margin_params[1], fixed_start=True)

@api_utils.transform
def api(x):
    return wrap.f(x)
print("starting point & reward", x0, y0)
print(f"initial margin {m0:,.0f}")

#  random queries for training GP
X, y = slicer.static_random_query(x0.size(-1), api, m0, N, q)

#  gp evaluation
records = gp_evaluation.evaluation(X, y)
kernel, mode, opt, _, _ = gp_evaluation.result_and_save(records, data_folder, 1)
gp_name, gp_params["mode"], gp_params["opt"] = kernel, mode, opt
print("")
print(f"gp is {kernel}, {mode}, {opt}")

#  experiment 1: exploration- exploitation trade-off
best_beta = exp_helper.exp1(gp_name, gp_params, hyper_params, data_folder, 1, T, x0, y0, m0, api, runs)
print("")
print(f"best beta {best_beta:.1f}")

#  experiment 2: UCB_EI && their parallelism
exp_helper.exp2(gp_name, gp_params, hyper_params, data_folder, 1, T, x0, y0, m0, api, runs, best_beta)

#  experiment 3: KG
exp_helper.exp3(gp_name, gp_params, hyper_params, data_folder, 1, T, x0, y0, m0, api, runs)

#  experiment 4: warmups
exp_helper.exp4(gp_name, gp_params, hyper_params, data_folder, 1, T, x0, y0, m0, api, runs)
    

In [11]:
i

2

In [ ]:
wrap.df(wrap.x0)

In [ ]:
%%time
margins_init = wrap.f(wrap.x0)
m0 = margins_init['margin'][0]
b0 = margins_init['buffer'][0]
print(f"${m0:,.0f}, buffer is: ${b0:,.0f}")